In [2]:
from json import load
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import WebBaseLoader

load_dotenv()
openai_api_key= os.getenv("OPENAI_API_KEY")
supabase_url= os.getenv("SUPABASE_URL")
supabase_key= os.getenv("SUPABASE_KEY")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
class DocumentLoaderHelper:
    def __init__(self, file_path):
        self.file_path = file_path

    def get_urls_from_json(self) -> list[str]:

        urls = []
        with open(self.file_path, 'r', encoding='utf-8') as file:
            data = load(file)
            if not isinstance(data, list):
                print(f"Error: The '{self.filepath}' is not a JSON list.")
                return []

            for source_group in data:
                if source_group.get('active') and isinstance(source_group.get('urls'), list):
                    for url in source_group['urls']:
                        if isinstance(url, str) and url.strip().startswith('http'):
                            urls.append(url.strip())
        return urls

Peter Atia site

In [ ]:
import bs4

web_loader = WebBaseLoader(web_path= DocumentLoaderHelper('config/web_sources.json').get_urls_from_json(), 
                           bs_kwargs={
                               "parse_only": bs4.SoupStrainer(
                                   class_=("entry-content")
                               ),
    },
    bs_get_text_kwargs={
        "separator": " ",
        "strip": True })

documents = []
async for document in web_loader.alazy_load():
    documents.append(document)

print(f"Loaded {len(documents)} documents")
print(documents)

Fetching pages: 100%|##########| 24/24 [00:09<00:00,  2.62it/s]


Loaded 24 documents
[Document(metadata={'source': 'https://peterattiamd.com/longevity-course/lesson1/'}, page_content='Lesson 1 What Exactly is “Longevity”? Longevity, as I define it, goes beyond the simple measure of how many years you get to live. It’s not just about adding time to your life; it’s about adding life to your time . When most people think of longevity, they envision some magical formula or elixir that promises to keep them alive forever, or at least well past 100 years. But, in reality, longevity is far more nuanced and involves optimizing both how long ( lifespan ) and how well ( healthspan ) you live. It’s about living longer while maintaining your health, fulfillment, and independence for as long as possible. This dual focus shifts our thinking from merely extending lifespan to extending and maximizing healthspan , ensuring that our extra years are not spent simply enduring, but truly living. My approach to longevity is firmly rooted in science but also involves a bi

Document Embedding

In [ ]:
from supabase import create_client, Client
from langchain_openai import OpenAIEmbeddings

db = create_client(supabase_url, supabase_key)
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

